Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. it contains algoritms that search in sets of vectors of any size , up to ones that possibly do not fit in Ram .it also containers supporting code for evaluation and parameter tuning 

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter



loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs = text_splitter.split_documents(documents)
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [ ]:
embeddings = OllamaEmbeddings(model="gemma:2b")
db = FAISS.from_documents(docs,embeddings)


In [6]:
db

In [8]:
## querying
query="How does the speaker describe the desired outcome of the war"
docs = db.similarity_search(query)
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

# As a Retriever
we can also convert the vectorstore into a Retriver class. This allows us to easily use it in other LangChain methods largely work with retrievers

In [9]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is  similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them . The returned distance score is L2 distance. Threfore, a lower score id better

In [11]:
docs_and_score = db.similarity_search_with_score(query)


In [12]:
docs_and_score

[(Document(id='665c39ae-be91-4753-a221-eff92b1a14e2', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(4981.128)),
 (Document(id='ff99e1dc-ac28-42fb-8446-571f92d7c20a', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that 

In [13]:
# save the file
db.save_local("faiss_index")

In [16]:
# load db from local vectordb

new_df = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs= new_df.similarity_search(query)
docs

[Document(id='665c39ae-be91-4753-a221-eff92b1a14e2', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='ff99e1dc-ac28-42fb-8446-571f92d7c20a', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a pat